In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import types

import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, date, timedelta

In [2]:
load_dotenv()
credentials_location = os.getenv('GCP_CREDENTIALS_LOCATION')
gcp_bucket_name = os.getenv('GCP_BUCKET')
gcp_temporary_bucket = os.getenv('GCP_TEMP_BUCKET')
PROJECT_ID = os.getenv('GCP_PROJECT_ID')

In [3]:
def connection_setup():
    """
    This function sets up the spark connection and reads the data from cloud storage
    """
    #jar files
    jar_1 = "/Users/Manu/lib/spark-bigquery-with-dependencies_2.12-0.24.2.jar"
    jar_2 = "/Users/Manu/lib/gcs-connector-hadoop3-2.2.5.jar"
    data_date = datetime.today().strftime('%Y-%m-%d')
    
    
    # configuration
    conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", f'{jar_1}, {jar_2}') \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)
    
    # context
    sc = SparkContext(conf=conf)
    hadoop_conf = sc._jsc.hadoopConfiguration()
    hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
    hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
    hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
    hadoop_conf.set("fs.gs.auth.service.account.enable", 'true')
    
    # Session
    spark = SparkSession.builder \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2", conf=sc.getConf()) \
    .getOrCreate()
    
    # Read the data
    cot = spark.read \
        .option('header', 'true') \
        .csv(f'gs://{gcp_bucket_name}/raw/*') #_{data_date}.txt')
    
    return cot

In [4]:
cot = connection_setup()

22/06/10 17:44:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def get_latest_data(cot):
    """
    This function fetches the most recent unprocessed data
    """
#     report_date = date.today() - timedelta(days = 9)
#     cot = cot.withColumn('Report_Date_as_YYYY-MM-DD', cot['Report_Date_as_YYYY-MM-DD'].cast(types.DateType()))
#     cot = cot.filter(cot['Report_Date_as_YYYY-MM-DD'] >= report_date)
    
    return cot

In [6]:
cot_filter = get_latest_data(cot)

In [7]:
cot_filter.count()
#cot_filter.select(['Market_and_Exchange_Names', 'Report_Date_as_YYYY-MM-DD', 'Open_Interest_All']).show()

24551

In [8]:
def clean_columns(cot):        
    # extract the numeric part of the contracts
    # drop the As_of_Date_In_Form_YYMMDD column
    cot_cols = cot.withColumn('Contract_Units', F.substring(cot['Contract_Units'], 19,7))\
                .drop('As_of_Date_In_Form_YYMMDD')
    
    # Make all columns lower case
    for col in cot_cols.columns:
        cot_cols = cot_cols.withColumnRenamed(col, col.lower())
    
    return cot_cols
    

In [9]:
cot_df = clean_columns(cot)

In [10]:
def add_schema(cot_df):
    desired_schema = [
    types.StringType(),types.DateType(),types.StringType(),types.StringType(),types.StringType(),types.StringType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),
    types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),
    types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),types.IntegerType(),
    types.IntegerType(),types.StringType(),types.StringType(),types.FloatType(),types.FloatType(),types.FloatType(),
    types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.FloatType(),types.StringType(),
    types.StringType(),types.StringType(),types.StringType(),types.StringType(),types.StringType()
    ]
    
    for idx, col in enumerate(cot_df.columns):
        for idx_s, dtype in enumerate(desired_schema):
            if idx == idx_s:
                cot_df = cot_df.withColumn(col, cot_df[col].cast(dtype))
            else:
                pass
            
    return cot_df

In [11]:
cot_schema = add_schema(cot_df)

In [12]:
def write_to_parquet(cot_schema):
    cot_schema.write.parquet(f'gs://{gcp_bucket_name}/cleaned/pq', mode='overwrite')
    return cot_schema

In [13]:
write_to_parquet(cot_schema)

22/06/10 17:45:29 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/06/10 17:45:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Market_and_Exchange_Names, Report_Date_as_MM_DD_YYYY, CFTC_Contract_Market_Code, CFTC_Market_Code, CFTC_Region_Code, CFTC_Commodity_Code, Open_Interest_All, Dealer_Positions_Long_All, Dealer_Positions_Short_All, Dealer_Positions_Spread_All, Asset_Mgr_Positions_Long_All, Asset_Mgr_Positions_Short_All, Asset_Mgr_Positions_Spread_All, Lev_Money_Positions_Long_All, Lev_Money_Positions_Short_All, Lev_Money_Positions_Spread_All, Other_Rept_Positions_Long_All, Other_Rept_Positions_Short_All, Other_Rept_Positions_Spread_All, Tot_Rept_Positions_Long_All, Tot_Rept_Positions_Short_All, NonRept_Positions_Long_All, NonRept_Positions_Short_All, Change_in_Open_Interest_All, Change_in_Dealer_Long_All, Change_in_Dealer_Short_All, Chan

DataFrame[market_and_exchange_names: string, report_date_as_yyyy-mm-dd: date, cftc_contract_market_code: string, cftc_market_code: string, cftc_region_code: string, cftc_commodity_code: string, open_interest_all: int, dealer_positions_long_all: int, dealer_positions_short_all: int, dealer_positions_spread_all: int, asset_mgr_positions_long_all: int, asset_mgr_positions_short_all: int, asset_mgr_positions_spread_all: int, lev_money_positions_long_all: int, lev_money_positions_short_all: int, lev_money_positions_spread_all: int, other_rept_positions_long_all: int, other_rept_positions_short_all: int, other_rept_positions_spread_all: int, tot_rept_positions_long_all: int, tot_rept_positions_short_all: int, nonrept_positions_long_all: int, nonrept_positions_short_all: int, change_in_open_interest_all: int, change_in_dealer_long_all: int, change_in_dealer_short_all: int, change_in_dealer_spread_all: int, change_in_asset_mgr_long_all: int, change_in_asset_mgr_short_all: int, change_in_asset_

In [16]:
def write_to_bigquery(cot_schema):
    cot_schema.write \
                .format('bigquery') \
                .option('project', PROJECT_ID) \
                .option('parentProject', PROJECT_ID) \
                .option('table', 'committment_of_traders.cot') \
                .option("temporaryGcsBucket",f"{gcp_temporary_bucket}") \
                .mode('overwrite') \
                .save()
    

In [17]:
write_to_bigquery(cot_schema)

22/06/10 17:53:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Market_and_Exchange_Names, Report_Date_as_MM_DD_YYYY, CFTC_Contract_Market_Code, CFTC_Market_Code, CFTC_Region_Code, CFTC_Commodity_Code, Open_Interest_All, Dealer_Positions_Long_All, Dealer_Positions_Short_All, Dealer_Positions_Spread_All, Asset_Mgr_Positions_Long_All, Asset_Mgr_Positions_Short_All, Asset_Mgr_Positions_Spread_All, Lev_Money_Positions_Long_All, Lev_Money_Positions_Short_All, Lev_Money_Positions_Spread_All, Other_Rept_Positions_Long_All, Other_Rept_Positions_Short_All, Other_Rept_Positions_Spread_All, Tot_Rept_Positions_Long_All, Tot_Rept_Positions_Short_All, NonRept_Positions_Long_All, NonRept_Positions_Short_All, Change_in_Open_Interest_All, Change_in_Dealer_Long_All, Change_in_Dealer_Short_All, Change_in_Dealer_Spread_All, Change_in_Asset_Mgr_Long_All, Change_in_Asset_Mgr_Short_All, Change_in_Asset_Mgr_Spread_All, Change_in_Lev_Money_Long_All, Change_in_Lev_Money_Short_All, C

In [ ]:
datetime.today().strftime('%Y-%m-%d')

In [ ]:
!pip install pipe

In [ ]:
import pipe

In [ ]:
help(pipe)

In [ ]:
def first(x):
    return x+4

def second(x):
    return x - 4



In [ ]:
frst = pipe.Pipe(first)
second = pipe.Pipe(second)

In [ ]:
1 | second 

In [ ]:
lst = ['first', 'second', 'third']

In [ ]:
idx = lst.index('first')

In [ ]:
lst.insert(idx + 1, 'first_1')

In [ ]:
lst